
# **CNN with CIFAR10**

이 튜토리얼에서는 CIFAR 이미지를 분류하기 위한 간단한 CNN(Convolutional Neural Network) 교육을 시연합니다. 본 튜토리얼에서는 Keras Sequential API를 사용하기 때문에 모델을 만들고 교육하는 데 몇 줄의 코드만 사용합니다.

## Import TensorFlow

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

## Download and prepare the CIFAR10 dataset

CIFAR10 데이터 세트에는 10개 클래스에 60,000개의 컬러 영상이 포함되며 각 클래스에 6,000개의 이미지가 있습니다.<br> 데이터 세트는 50,000개의 교육 이미지와 10,000개의 테스트 이미지로 나뉩니다.<br> 클래스는 상호 배타적이며 서로 겹치지 않습니다.

In [ ]:
from sklearn.model_selection import train_test_split
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Nomalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

# train_images, train_labels 데이터를 validation데이터로 나누기
train_images, x_val, train_labels, y_val = train_test_split(train_images, train_labels, test_size=0.2, random_state=2021)


In [ ]:
print(train_images.shape)
print(x_val.shape)
print(train_labels.shape)
print(y_val.shape)
print(test_images.shape)
print(test_labels.shape)

## Verify the data

데이터 집합이 올바르게 표시되는지 확인하려면 교육 세트의 처음 25개 이미지를 플롯하고 각 이미지 아래에 클래스 이름을 표시합니다.

In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10, 10))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i])
    # The CIFAR labels happen to be arrays
    # which is why you need the extra index
    plt.xlabel(class_names[train_labels[i][0]])
plt.show()

## Create the convolutional base

아래 코드 6줄은 Conv2D 및 MaxPooling 2D 도면층의 스택이라는 공통 패턴을 사용하여 컨볼루션 베이스를 정의합니다.

- 입력으로 CNN은 배치 크기를 무시하고 모양(image_height, image_width, color_channel)의 텐서를 사용합니다.
- 이러한 치수를 처음 사용하는 경우 color_channels는 (R,G,B)를 참조합니다.
- 이 예에서는 CIFAR 이미지의 형식인 형상 입력(32, 32, 3)을 처리하도록 CNN을 구성합니다.
- input_shape 인수를 첫 번째 도면층에 전달하여 이 작업을 수행할 수 있습니다.

In [ ]:
model = models.Sequential()
# conv의 필터개수는 32, 필터사이즈 3 X 3
# input size는 32 X 32 X 3
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2))) # pooling 사이즈 2 X 2
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [ ]:
# 지금까지 모델의 아키텍처를 표시
model.summary()

- 첫번째 conv의 param은  3(필터사이즈) X 3(필터사이즈) X 3(입력채널 수) X 32(필터의 개수) + 32(Bias) = 896
- Conv가 진행되면 input사이즈는 스트라이드가 1이기때문에 -2가 줄어듬
- max_pooling의 사이즈가 2 X 2이기때문에 image의 사이즈는 1/2로 축소됨


위에서 모든 Conv2D 및 MaxPooling 2D 레이어의 출력이 형상(높이, 폭, 채널)의 3D 텐서임을 확인할 수 있습니다.<br> 너비와 높이 치수는 네트워크에서 더 깊이 들어갈수록 축소되는 경향이 있습니다.<br> 각 Conv2D 계층의 출력 채널 수는 첫 번째 인수(예: 32 또는 64)에 의해 제어됩니다.<br> 일반적으로 너비와 높이가 줄어들면 (계산적으로) 각 Conv2D 계층에 더 많은 출력 채널을 추가할 수 있습니다.

## Add Dense layers on top

모델을 완료하려면 마지막 출력 텐서를 컨볼루션 베이스(4, 4, 64))에서 하나 이상의 조밀한 레이어로 공급하여 분류를 수행합니다.<br> 밀도가 높은 레이어는 벡터를 입력(1D)으로 사용하고, 전류 출력은 3D 텐서입니다.<br> 먼저 3D 출력을 1D로 평탄화(또는 롤링 해제)한 다음 맨 위에 하나 이상의 밀도 레이어를 추가합니다.<br> CIFAR에는 10개의 출력 클래스가 있으므로 10개의 출력과 함께 최종 밀도 레이어를 사용합니다.

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10)) # 마지막 출력은 클래스가 10이기 때문에 10으로 출력

In [ ]:
# 다음은 전체 모델 아키텍처
model.summary()

- Dense layer의 입력수는 flatten을 거쳐 1024
- Dense layer의 parm의 수는 1024 X 64(노드의 수) + 64(bias)
- 마지막 출력의 Parm은 64(input) X 10(노드의수) + 10

네트워크 요약에 따르면 (4, 4, 64) 출력은 두 개의 Dense layer를 거치기 전에 형상 벡터(1024)로 flatten 해졌습니다.

## Compile and train the model

In [ ]:
model.compile(optimizer = 'adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), # 확률이 아닌 값이 나오게 된다면, logit=True라고 표현
              metrics=['accuracy'])                                                 # 해당 클래스의 범위에서의 확률을 출력한다면, 이를 logit=False라고 표현
                                                                                    # 출력값을 log처리 하느냐 아니냐 (softmax 함수를 거치면 False)
history = model.fit(train_images, train_labels, epochs=10,
                    validation_data =(x_val, y_val))

## Evaluate the model


In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

print(test_acc)

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)

y_pred = model.predict(test_images)

import seaborn as sns
import pandas as pd
import numpy as np

test_labels = np.squeeze(test_labels, axis=None)
y_pred = np.array([np.argmax(y) for y in y_pred])
df = pd.DataFrame({'test_labels' : test_labels, 'y_pred' : y_pred}, columns=['test_labels', 'y_pred'])

print(df)

for col in df.columns[0]:
    plt.figure(figsize=(15,5))
    plt.title('{}'.format(col), size=20)
    plt.plot(df['test_labels'], df['{}'.format(test_labels)],'-o', label='true')
    plt.plot(df['y_pred'], df['{}'.format(y_pred)],'-o', label='predict')
    plt.xticks(rotation=20)

    plt.xlabel('date')
    plt.ylabel('{}'.format(col))
    plt.grid()
    plt.legend(loc='upper left')


# plt.plot(test_labels, color='green', marker='o')
# plt.plot(y_pred, color='red', marker='o')	# line 그래프를 그립니다
# plt.legend(['test_labels', 'y_pred'])
# plt.show()